# Video to Minutes of Meeting

**Tags**: #tool -> hastags of the topics the notebook is about, as text starting with the name of the tool

**Author:** [Maxime Jublou](https://www.linkedin.com/in/maximejublou) -> name and social profile link of the author(s)

**Last update:** YYYY-MM-DD (Created: YYYY-MM-DD) -> The last update date refers to when the notebook was last edited, while the created date corresponds to when the notebook was initially merged.

**Description:** This notebook demonstrates how to ... -> a one-liner explaining the benefits of the notebooks for the user, as text.

**References:** list of references and websites utilized in the creation of this notebook
- [Naas Documentation](https://site.naas.ai/)

## Input

### Install requirements

In [ ]:
!pip install --user google-auth==2.22.0 google-auth-oauthlib==0.4.6 google-auth-httplib2==0.1.0 google-api-python-client==2.43.0 openai==0.27.8 nltk==3.8.1

### Import libraries

In [ ]:
import naas
import markdown

from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build

from googleapiclient.http import MediaIoBaseDownload
import io

import openai

import os
import subprocess

from glob import glob

import concurrent.futures
import nltk
nltk.download('punkt')

### Setup variables

In [ ]:
email_to = [
    "your email address"
]

# Id of the folder from which we should compute MoMs
drive_folder_id = ""

# Location of GCP service account that will be used.
drive_service_account_file = 'service_account.json'

openai.api_key = naas.secret.get('OPENAI_KEY')
audio_model = "whisper-1"
text_model = "gpt-3.5-turbo-16k"

## Model

### Filesystem functions

In [ ]:
def remove_existing_video_audio_files():
    for f in glob('*.mp3') + glob('*.mp4'):
        print(f'🗑️ Removing {f}')
        os.remove(f)

### Google Drive functions

In [ ]:
creds = Credentials.from_service_account_file(drive_service_account_file)
service = build('drive', 'v3', credentials=creds)

#### List files in Drive

In [ ]:
def list_files():
    # Call the Drive v3 API
    results = service.files().list(
        q=f"'{drive_folder_id}' in parents", fields="nextPageToken, files(id, name)").execute()
    items = results.get('files', [])
    
    return items
    return [{i['id']: i['name']} for i in items]
# list_files()

#### Check if file is a video

In [ ]:
def is_video_file(file_id):
    # Retrieve the file metadata
    file = service.files().get(fileId=file_id, fields='mimeType').execute()

    # Check if the file is a video
    mime_type = file.get('mimeType', '')
    return mime_type.startswith('video/')

#### Get labels from files 

In [ ]:
def get_labels_from_file(file_id):
    # Retrieve the file with the properties field
    file = service.files().get(fileId=file_id, fields='properties').execute()

    # Get the labels from the properties field
    labels = file.get('properties', {})

    return labels

#### Add labels to file

In [ ]:
def add_labels_to_file(file_id, labels):
    # Update the file with the new labels
    service.files().update(fileId=file_id, body={'properties': labels}).execute()

#### Download file

In [ ]:
def download_file(file_id, filepath):
    request = service.files().get_media(fileId=file_id)
    fh = io.FileIO(filepath, 'wb')
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print("Download %d%%." % int(status.progress() * 100))

### FFMPEG functions

#### Download ffmpeg

In [ ]:
def install_ffmpeg():
    ffmpeg_path = os.path.join(os.getcwd(), 'ffmpeg_dir/ffmpeg')
    if os.path.isfile(ffmpeg_path):
        return ffmpeg_path
    
    
    if not os.path.isfile('ffmpeg-git-amd64-static.tar.xz'):
        !wget https://johnvansickle.com/ffmpeg/builds/ffmpeg-git-amd64-static.tar.xz
    
    if not os.path.isdir('ffmpeg_dir'):
        !mkdir ffmpeg_dir
    
    if os.path.isfile('ffmpeg-git-amd64-static.tar.xz') and os.path.isdir('ffmpeg_dir'):
        !tar xf ffmpeg-git-amd64-static.tar.xz -C ffmpeg_dir && mv ./ffmpeg_dir/*/* ffmpeg_dir/ && chmod +x ffmpeg_path
        
    return ffmpeg_path

#### Convert mp4 to mp3

The goal is to reduce the size of the file

In [ ]:
def convert_mp4_to_mp3(ffmpeg_path, input_file, output_file):
    command = f'{ffmpeg_path} -i {input_file} -vn -ab 128k -ar 44100 -y {output_file}'
    e = subprocess.run(command, shell=True)
    return e

#### Split audio file in small chunks

In [ ]:
def split_audio(ffmpeg_path, input_file, output_prefix, segment_time):
    command = f'{ffmpeg_path} -i {input_file} -f segment -segment_time {segment_time} -c copy {output_prefix}%03d.mp3'
    subprocess.run(command, shell=True)
    return sorted(glob(f'{output_prefix}*.mp3'))

### LLM functions

#### Get transcription for each chunks

In [ ]:
def _transcribe_audio(audio_file_path):
    print(f'⚙️ Transcribing {audio_file_path}')
    with open(audio_file_path, 'rb') as audio_file:
        transcription = openai.Audio.transcribe(audio_model, audio_file)
    
    print(f'✅ Transcribing {audio_file_path} done')
    
    return {
        'filename': audio_file_path,
        'transcript': transcription['text']
    }

def transcribe_audio(chunks):
    with concurrent.futures.ThreadPoolExecutor(max_workers=len(chunks)) as executor:
        futures = [executor.submit(_transcribe_audio, c) for c in chunks]

        results = [future.result() for future in concurrent.futures.as_completed(futures)]
        transcript = " ".join([c['transcript'] for c in sorted(results, key=lambda x: x['filename'])])
        return transcript

#### Split text in chunks

In [ ]:
def split_text(text, chunk_size):
    sentences = nltk.sent_tokenize(text)
    chunks = []
    current_chunk = ""
    current_chunk_size = 0

    for sentence in sentences:
        sentence_size = len(nltk.word_tokenize(sentence))
        if current_chunk_size + sentence_size <= chunk_size:
            current_chunk += sentence + " "
            current_chunk_size += sentence_size
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + " "
            current_chunk_size = sentence_size

    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks


#### Summary extraction

In [ ]:
def abstract_summary_extraction(transcription):
    response = openai.ChatCompletion.create(
        model=text_model,
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": "You are a highly skilled AI trained in language comprehension and summarization. I would like you to read the following text and summarize it into a concise abstract paragraph. Aim to retain the most important points, providing a coherent and readable summary that could help a person understand the main points of the discussion without needing to read the entire text. Please avoid unnecessary details or tangential points."
            },
            {
                "role": "user",
                "content": transcription
            }
        ]
    )

    return (response['choices'][0]['message']['content'])

#### Key points extraction

In [ ]:
def key_points_extraction(transcription):
    response = openai.ChatCompletion.create(
        model=text_model,
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": "You are a proficient AI with a specialty in distilling information into key points. Based on the following text, identify and list the main points that were discussed or brought up. These should be the most important ideas, findings, or topics that are crucial to the essence of the discussion. Your goal is to provide a list that someone could read to quickly understand what was talked about."
            },
            {
                "role": "user",
                "content": transcription
            }
        ]
    )
    
    return response['choices'][0]['message']['content']

#### Action item extraction

In [ ]:
def action_item_extraction(transcription):
    response = openai.ChatCompletion.create(
        model=text_model,
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": "You are an AI expert in analyzing conversations and extracting action items. Please review the text and identify any tasks, assignments, or actions that were agreed upon or mentioned as needing to be done. These could be tasks assigned to specific individuals, or general actions that the group has decided to take. Please list these action items clearly and concisely."
            },
            {
                "role": "user",
                "content": transcription
            }
        ]
    )
    
    
    return response['choices'][0]['message']['content']

#### Sentiment analysis
The sentiment_analysis function analyzes the overall sentiment of the discussion. It considers the tone, the emotions conveyed by the language used, and the context in which words and phrases are used. For tasks which are less complicated, it may also be worthwhile to try out gpt-3.5-turbo in addition to gpt-4 to see if you can get a similar level of performance. It might also be useful to experiment with taking the results of the sentiment_analysis function and passing it to the other functions to see how having the sentiment of the conversation impacts the other attributes.

In [ ]:
def sentiment_analysis(transcription):
    response = openai.ChatCompletion.create(
        model=text_model,
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": "As an AI with expertise in language and emotion analysis, your task is to analyze the sentiment of the following text. Please consider the overall tone of the discussion, the emotion conveyed by the language used, and the context in which words and phrases are used. Indicate whether the sentiment is generally positive, negative, or neutral, and provide brief explanations for your analysis where possible."
            },
            {
                "role": "user",
                "content": transcription
            }
        ]
    )
    
    return response['choices'][0]['message']['content']

#### Summarizing and analyzing the transcript

In [ ]:
def meeting_minutes(transcription):
    chunks = split_text(transcription, 10000)
    
    print('⚙️ Starting summary extraction')
    abstract_summary = abstract_summary_extraction(' '.join([abstract_summary_extraction(chunk) for chunk in chunks]))
    print('⚙️ Starting key points extraction')
    key_points = key_points_extraction(' '.join([key_points_extraction(chunk) for chunk in chunks]))
    print('⚙️ Starting action items extraction')
    action_items = action_item_extraction(' '.join([action_item_extraction(chunk) for chunk in chunks]))
    print('⚙️ Starting sentiment extraction')
    sentiment = abstract_summary_extraction(' '.join([sentiment_analysis(chunk) for chunk in chunks]))
    
    return {
        'abstract_summary': abstract_summary,
        'key_points': key_points,
        'action_items': action_items,
        'sentiment': sentiment
    }

### Markdown functions

In [ ]:
def minute_to_markdown(minutes):
    md = ""

    for key, value in minutes.items():
        # Replace underscores with spaces and capitalize each word for the heading
        heading = ' '.join(word.capitalize() for word in key.split('_'))
        md += '# ' + heading + '\n'
        md += value + '\n'
        # Add a line break between sections
        md += '\n'
    
    return md

## Output

### Run

In [ ]:
def run(debug=True):
    files = list_files()
    
    for file in files:
        if is_video_file(file['id']):
            labels = get_labels_from_file(file['id'])
            if 'minute_done_and_sent' in labels and labels['minute_done_and_sent'] == 'true':
                if debug: print(f'⏭️ Skipping {file} already done.')
                continue
            
            if debug: print("... Removing existing audio files")
            remove_existing_video_audio_files()
            
            if debug: print("... Downloading file")
            download_file(file['id'], 'out.mp4')
            
            if debug: print("... Installing ffmpeg")
            ffmpeg_path = install_ffmpeg()
            
            if debug: print("... Converting mp4 to mp3")
            convert_mp4_to_mp3(ffmpeg_path, 'out.mp4', 'out.mp3')
            
            if debug: print("... Splitting audio in chunks")
            chunks = split_audio(ffmpeg_path, 'out.mp3', 'chunk', 300)
            
            if debug: print("... Getting transcript from audio files")
            transcript = transcribe_audio(chunks)
            
            if debug: print("... Converting transcript to minutes")
            minutes = meeting_minutes(transcript)
            
            if debug: print("... Generating Mardown from minutes")
            md_content = minute_to_markdown(minutes)
            if debug: print(md_content)
            
            naas.notification.send(email_to=email_to, subject=f"Minutes for recording {file['name']} completed!", html=f"""
            Minutes for recording: https://drive.google.com/file/d/{file['id']}/view
            
            {markdown.markdown(md_content)}
            """)
            
            labels['minute_done_and_sent'] = 'true'
            add_labels_to_file(file['id'], labels)
            
        else:
            if debug: print(f'⏭️ {file} is not a video file.')
            
    
    
run(debug=True)

### Schedule

Let's schedule to receive the MoM automatically when a recording is available.

In [ ]:
naas.scheduler.add(cron="*/5 * * * *")